# SAM (Segment Anything Model) 分割教程

> 完整演示SAM模型的各种使用方式

**学习目标**：
- 掌握SAM的点/框提示分割
- 学会自动分割整图
- 了解SAM与CLIP的结合

**预计时间**: 45-60分钟

In [ ]:
# 导入必要的库
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
from pathlib import Path

from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

## 1. 加载SAM模型

In [ ]:
# 加载模型
model_type = "vit_b"
checkpoint_path = "../models/sam/sam_vit_b_01ec64.pth"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用设备: {device}")

sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
sam.to(device=device)
predictor = SamPredictor(sam)

print("✅ SAM模型加载完成")

## 2. 准备示例图像

In [ ]:
# 准备示例图像
import os

# 方案1：尝试从网络下载示例图像
image_url = "https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg"
image_path = "sample_image.jpg"

try:
    if not os.path.exists(image_path):
        print("📥 下载示例图像...")
        urllib.request.urlretrieve(image_url, image_path)
        print("✅ 下载成功")
except Exception as e:
    print(f"⚠️ 下载失败: {e}")
    print("生成测试图像...")
    # 方案2：生成一个简单的测试图像
    test_image = np.random.randint(0, 255, (400, 600, 3), dtype=np.uint8)
    # 绘制一些形状
    import cv2
    cv2.rectangle(test_image, (100, 100), (500, 300), (255, 0, 0), -1)
    cv2.circle(test_image, (300, 200), 50, (0, 255, 0), -1)
    Image.fromarray(test_image).save(image_path)
    print("✅ 生成测试图像")

# 加载图像
image = Image.open(image_path).convert("RGB")
image = np.array(image)

# 显示图像
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title("原始图像")
plt.axis('off')
plt.show()

# 设置到预测器
predictor.set_image(image)
print("✅ 图像已设置")

## 3. 点提示分割

In [ ]:
# 定义点提示
input_point = np.array([[image.shape[1]//2, image.shape[0]//2]])
input_label = np.array([1])  # 1 = 前景

# 预测
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)

print(f"生成了 {len(masks)} 个候选掩码")
print(f"IoU分数: {scores}")

## 4. 框提示分割

In [ ]:
# 定义边界框
h, w = image.shape[:2]
margin = 0.2
input_box = np.array([
    int(w * margin), int(h * margin),
    int(w * (1 - margin)), int(h * (1 - margin))
])

# 预测
masks, scores, logits = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_box[None, :],
    multimask_output=False,
)

print(f"分割完成, IoU: {scores[0]:.3f}")

## 5. 自动分割

In [ ]:
# 创建自动掩码生成器
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=16,
    pred_iou_thresh=0.86,
    stability_score_thresh=0.92,
    min_mask_region_area=100,
)

print("🔍 正在自动分割...")
masks = mask_generator.generate(image)
print(f"✅ 完成！找到 {len(masks)} 个物体")

## 总结

本教程演示了SAM的三种主要使用方式：

1. **点提示分割** - 最简单灵活
2. **框提示分割** - 稳定准确
3. **自动分割** - 无需提示

### 练习任务

1. 在自己的图像上测试SAM
2. 尝试不同的提示组合
3. 调整自动分割参数
4. 与CLIP结合实现语义分割

### 参考资源

- [SAM官方仓库](https://github.com/facebookresearch/segment-anything)
- [SAM论文](https://arxiv.org/abs/2304.02643)
- [SAM详解文档](../docs/01-模型调研与选型/05-SAM模型详解.md)

🎉 恭喜完成本教程！